## Importando Bibliotecas

In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report

### Feito leitura dos dados e verificado e definido o target do atual trabalho

#####Target Definido = Class Label
#####1 -> Fishing / 0 -> Legítimo

#### Utilização do método otimização de hiperparâmetros com o modelo de Decision Tree.

In [5]:
df = pd.read_csv('/content/Phishing_Legitimate_full.csv', index_col=False)
df.head()

#1 -> Fishing / #0 -> Legítimo

,id,NumDots,SubdomainLevel,PathLevel,UrlLength,NumDash,NumDashInHostname,AtSymbol,TildeSymbol,NumUnderscore,...,IframeOrFrame,MissingTitle,ImagesOnlyInForm,SubdomainLevelRT,UrlLengthRT,PctExtResourceUrlsRT,AbnormalExtFormActionR,ExtMetaScriptLinkRT,PctExtNullSelfRedirectHyperlinksRT,CLASS_LABEL
0,1,3,1,5,72,0,0,0,0,0,...,0,0,1,1,0,1,1,-1,1,1
1,2,3,1,3,144,0,0,0,0,2,...,0,0,0,1,-1,1,1,1,1,1
2,3,3,1,2,58,0,0,0,0,0,...,0,0,0,1,0,-1,1,-1,0,1
3,4,3,1,6,79,1,0,0,0,0,...,0,0,0,1,-1,1,1,1,-1,1
4,5,3,0,4,46,0,0,0,0,0,...,1,0,0,1,1,-1,0,-1,-1,1


#### Criando matriz de correlação:
####Etapa importante para verificarmos qual a correlação das outra features com o target que foi definido.

In [6]:
pd.set_option('display.max_columns', 50)
corr_mat = df.corr(method='pearson')
corr_mat

,id,NumDots,SubdomainLevel,PathLevel,UrlLength,NumDash,NumDashInHostname,AtSymbol,TildeSymbol,NumUnderscore,NumPercent,NumQueryComponents,NumAmpersand,NumHash,NumNumericChars,NoHttps,RandomString,IpAddress,DomainInSubdomains,DomainInPaths,HttpsInHostname,HostnameLength,PathLength,QueryLength,DoubleSlashInPath,NumSensitiveWords,EmbeddedBrandName,PctExtHyperlinks,PctExtResourceUrls,ExtFavicon,InsecureForms,RelativeFormAction,ExtFormAction,AbnormalFormAction,PctNullSelfRedirectHyperlinks,FrequentDomainNameMismatch,FakeLinkInStatusBar,RightClickDisabled,PopUpWindow,SubmitInfoToEmail,IframeOrFrame,MissingTitle,ImagesOnlyInForm,SubdomainLevelRT,UrlLengthRT,PctExtResourceUrlsRT,AbnormalExtFormActionR,ExtMetaScriptLinkRT,PctExtNullSelfRedirectHyperlinksRT,CLASS_LABEL
id,1.000000,-0.238066,0.027976,-0.234909,-0.046967,0.231091,-0.107170,0.009916,-0.084148,0.024811,0.042614,0.047733,-0.002700,-0.050400,-0.072220,0.065735,0.050651,-0.174331,-0.063880,0.081811,NaN,-0.109106,-0.012361,-0.015069,-0.023122,-0.214784,-0.127501,-0.245957,-0.010172,-0.085581,-0.274347,0.073639,0.096961,0.130747,-0.266362,-0.427392,0.007209,-0.041793,0.052504,0.305298,0.204099,-0.081004,0.000481,0.063732,-0.010836,-0.020932,-0.169502,-0.068570,0.465757,-0.866025
NumDots,-0.238066,1.000000,0.507992,0.079202,0.277380,-0.198769,0.093034,0.011425,0.109508,0.038068,0.022616,0.169283,0.255404,0.371642,0.211384,0.024590,-0.064631,0.158447,0.238473,0.009897,NaN,0.289482,0.000579,0.263039,0.002462,0.305793,0.164434,0.116726,-0.042685,0.050103,0.072103,-0.088802,-0.009604,-0.031990,0.033516,0.177703,-0.002490,0.050992,-0.031698,-0.117766,-0.117210,0.045291,-0.012244,-0.343927,-0.151756,0.047821,0.032276,0.035586,-0.153835,0.294111
SubdomainLevel,0.027976,0.507992,1.000000,0.007277,0.098579,-0.050018,0.246368,-0.005845,0.009518,-0.023593,-0.006003,-0.029706,-0.031766,-0.018054,0.051744,-0.044628,0.076544,-0.103343,0.522014,-0.019847,NaN,0.605218,-0.039307,-0.026179,0.012070,0.158365,0.019471,-0.063890,-0.079427,-0.033224,-0.040207,-0.020441,0.002922,0.029142,0.106631,-0.061013,-0.023893,0.133530,-0.010968,-0.027739,-0.001061,0.026432,-0.008830,-0.679687,-0.086943,0.059584,-0.030790,0.093623,-0.034866,0.043132
PathLevel,-0.234909,0.079202,0.007277,1.000000,0.390456,0.090503,0.027626,-0.005891,0.127291,0.015719,-0.043529,-0.140303,-0.104935,0.007947,0.233024,0.058466,0.250091,0.017895,0.014693,0.261468,NaN,0.013025,0.612786,-0.085555,0.020221,0.121805,0.087328,0.018365,-0.038542,0.082978,0.099618,0.007842,-0.012905,-0.033858,0.085679,0.028918,-0.004003,0.049318,0.014823,-0.061813,0.000701,-0.027880,-0.002279,-0.024815,-0.399710,-0.000903,0.040901,0.064222,-0.095786,0.229450
UrlLength,-0.046967,0.277380,0.098579,0.390456,1.000000,0.437313,0.111980,-0.003425,0.058233,0.350756,0.184947,0.476699,0.466038,0.227029,0.637872,0.036667,0.230698,0.094311,0.120201,0.227722,NaN,0.162770,0.643924,0.649204,0.000162,0.223492,0.026183,-0.095430,-0.045713,0.061362,-0.004030,0.048207,0.015086,0.023152,-0.090563,-0.099667,0.014242,0.087848,0.034980,0.042056,0.033192,-0.053222,-0.036766,-0.133533,-0.800096,0.002019,-0.025366,-0.000860,0.142819,-0.074493
NumDash,0.231091,-0.198769,-0.050018,0.090503,0.437313,1.000000,0.113200,-0.010139,-0.024395,-0.050195,-0.017938,-0.064746,-0.056693,-0.023398,0.121830,0.036594,0.248141,-0.056135,0.056684,0.315989,NaN,0.010045,0.605889,-0.013725,-0.001462,-0.081733,-0.043440,-0.070490,0.116288,0.086994,-0.091292,0.094711,0.077007,0.049723,-0.113173,-0.188701,-0.007831,0.063704,0.092634,0.181687,0.228154,-0.064450,-0.006314,-0.023363,-0.468798,-0.137093,-0.085238,-0.122579,0.184808,-0.372235
NumDashInHostname,-0.107170,0.093034,0.246368,0.027626,0.111980,0.113200,1.000000,-0.004409,-0.018043,-0.035170,-0.021648,-0.042379,-0.036415,-0.008395,0.120447,-0.089577,0.073574,-0.033672,0.542477,0.038602,NaN,0.550247,0.000450,-0.029753,0.065692,0.081098,0.013179,-0.025364,-0.078277,0.004301,0.011456,0.075694,-0.011151,-0.041690,0.177997,-0.019193,-0.009018,0.439129,-0.01

#### Drop de features
####Após visualização de matriz de correlação, foi feito o drop das features chaves como o ID e a Class_label (target).

In [7]:
#Drop do target e do index
X = df.drop(columns=['CLASS_LABEL', 'id'])
y = df['CLASS_LABEL']

#### Criação da Pipeline, com o StandardScaler e o model(DecisionTreeClassifier).

In [8]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', DecisionTreeClassifier(random_state=42))
])

#### Foi feito separação dos dados de treino e teste

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Utilizado Para otimização de Hiperparametro o Random Search

<b>Prós</b>:
- Mais eficiente do que Grid Search quando o espaço de hiperparâmetros é grande.
- Pode encontrar boas soluções com menos recursos computacionais.

<b>Contras</b>:
- Ao encontrar uma solução não ótima pode levar a resultados inconsistentes dado os parâmetros os quais foram testados


###### cv = Determina a estratégia de divisão de validação cruzada

In [10]:
#Criação de validação random search
param_random_grid = {
    'model__criterion': ['gini', 'entropy'],
    'model__max_features': list(range(1, 48)),
    'model__max_depth': [None, 4, 8, 10, 12],
    'model__class_weight': [None, 'balanced'],
    'model__min_samples_split': [2, 6, 10, 25]
}

random_search = RandomizedSearchCV(pipe, param_random_grid, random_state=42, n_iter=40, cv=5 ,n_jobs=-1)


#### Treinamento do modelo

In [11]:
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                             ('model',
                                              DecisionTreeClassifier(random_state=42))]),
                   n_iter=40, n_jobs=-1,
                   param_distributions={'model__class_weight': [None,
                                                                'balanced'],
                                        'model__criterion': ['gini', 'entropy'],
                                        'model__max_depth': [None, 4, 8, 10,
                                                             12],
                                        'model__max_features': [1, 2, 3, 4, 5,
                                                                6, 7, 8, 9, 10,
                                                                11, 12, 13, 14,
                                                                15, 16, 17, 18,
                                                                19, 20, 21, 22,
                                                                23, 24, 25, 26,
                                                                27, 28, 29, 30, ...],
                                        'model__min_samples_split': [2, 6, 10,
                                                                     25]},
                   random_state=42)

#### Após treinamento, foram encontrados os melhores hiperparâmetros através do uso de Random Seacrh.

In [12]:
random_search.best_params_

{'model__min_samples_split': 10,
 'model__max_features': 42,
 'model__max_depth': None,
 'model__criterion': 'entropy',
 'model__class_weight': 'balanced'}

#### Atribuição da predição

In [13]:
y_pred = random_search.predict(X_test)

#### Classification_report

In [14]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       988
           1       0.96      0.98      0.97      1012

    accuracy                           0.97      2000
   macro avg       0.97      0.97      0.97      2000
weighted avg       0.97      0.97      0.97      2000

